In [55]:
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio

In [56]:
class GradientDescent:
    def __init__(self, scheduler, epochs, f_samples, x0):
        """
        :param lr: learning rate
        """
        self.scheduler = scheduler
        self.weights =  x0
        self.epochs = epochs
        self.f_samples = f_samples


    def train(self, f_gradient, f, gifname):
        try:
            os.mkdir('gifs')
        except Exception:
            import shutil
            shutil.rmtree('gifs')
            os.mkdir('gifs')

        for i in range(self.epochs):
            lr = self.scheduler.step()
            self.weights = self.weights - lr * f_gradient(self.weights)

            if i % 1 == 0:
                plt.scatter(self.f_samples[0], self.f_samples[1], c='blue')
                plt.scatter(self.weights, f(self.weights), c='yellow')
                plt.title(f'Epoch: {i}')
                plt.xlabel('x')
                plt.ylabel('y')
                plt.savefig(f"gifs/{i}.jpg")
        plt.clf()

        with imageio.get_writer(f'{gifname}.gif', mode='I') as writer:
            for filename in sorted(os.listdir('gifs'), key=lambda x: int(x.split('.')[0])):
                image = imageio.imread(f"gifs/{filename}")
                writer.append_data(image)

In [57]:
f = lambda x: (x ** 3) / 3 - (x ** 2) / 2 - x - 1
f_grad = lambda weights: weights ** 2 - weights - 1
# f = lambda x: x ** 2 + 4
# f_grad = lambda weights: 2 * weights

In [58]:
xs = np.linspace(-5, 5, 1000)
ys = np.vectorize(f)(xs)

In [59]:
class Scheduler:
    def __init__(self, start_lr):
        self.start_lr = start_lr

In [60]:
class ConstantScheduler(Scheduler):
    def __init__(self, start_lr):
        Scheduler.__init__(self, start_lr)

    def step(self):
        return self.start_lr

In [61]:
class StepScheduler(Scheduler):
    def __init__(self, start_lr, scheduler_step, gamma):
        assert scheduler_step > 0

        Scheduler.__init__(self, start_lr)
        self.cur_step = 0
        self.scheduler_step = scheduler_step
        self.gamma = gamma

    def step(self):
        self.cur_step += 1
        if self.cur_step % self.scheduler_step == 0:
            self.start_lr = self.gamma * self.start_lr
        return self.start_lr

## Пункт 1 - Constat LR GD

In [62]:
const_scheduler = ConstantScheduler(1)
const_trainer = GradientDescent(const_scheduler, 50, (xs, ys), x0=np.array([4]))
const_trainer.train(f_grad, f, 'const_lr=1')

<Figure size 432x288 with 0 Axes>

![SegmentLocal](const_lr=1.gif "segment")

In [63]:
const_scheduler = ConstantScheduler(1e-2)
const_trainer = GradientDescent(const_scheduler, 50, (xs, ys), x0=np.array([4]))
const_trainer.train(f_grad, f, 'const_lr=1e-2')

<Figure size 432x288 with 0 Axes>

![SegmentLocal](const_lr=1e-2.gif "segment")

# Пункт 2 - Step LR GD

In [64]:
step_scheduler = StepScheduler(1, scheduler_step=15, gamma=0.1)
step_trainer = GradientDescent(step_scheduler, 50, (xs, ys), x0=np.array([4]))
step_trainer.train(f_grad, f, 'step')

<Figure size 432x288 with 0 Axes>

![SegmentLocal](step.gif "segment")

# Пункт 3 - Дихотомия

In [65]:
DEFAULT_EPS = 1e-3

In [66]:
def dichotomy(f, f_samples, gif_folder, gifname, eps=DEFAULT_EPS, plot=True):
    delta = eps / 3
    os.mkdir(gif_folder)
    def step(old_interval, cnt=0):
        a, b = old_interval
        mid = (a + b) / 2

        if plot:
            plt.scatter(f_samples[0], f_samples[1], c='blue')
            plt.scatter(mid, f(mid), c='yellow')
            plt.title(f'Epoch: {cnt}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.savefig(f"{gif_folder}/{cnt}.jpg")
            plt.clf()

            with imageio.get_writer(f'{gifname}.gif', mode='I') as writer:
                for filename in sorted(os.listdir(gif_folder), key=lambda x: int(x.split('.')[0])):
                    image = imageio.imread(f"{gif_folder}/{filename}")
                    writer.append_data(image)

        if b - a < eps:
            return mid

        x1, x2 = mid - delta, mid + delta
        fx1, fx2 = f(x1), f(x2)

        if fx1 < fx2:
            new_interval = (a, x2)
        elif fx1 > fx2:
            new_interval = (x1, b)
        else:
            new_interval = (x1, x2)

        return step(new_interval, cnt+1)

    return step

In [68]:
dichotomy_algo = dichotomy(f, (xs, ys), "dichotomy", "simple_dichotomy")

In [ ]:
dichotomy_algo([-1000, 1000])

![SegmentLocal](simple_dichotomy.gif "segment")